## Section 1 - Authenticing / Access credentials 

In [ ]:
import requests

# Listing Clouds
domain = "DOMO DOMIN"
access_token = "DOMO Access token"

response = requests.get(
    url=f"https://{domain}/api/query/v1/byos/accounts/basic",
    headers={"X-Domo-Developer-Token": access_token},
)

print(response.json())

## Section 2 - Getting the cloud_id from the JSON output answer

In [ ]:
# Make the right call when choosing the friendlyName which is the Domo Multi - cloud account 

import requests

# Listing Cloud for the Domo connection
domain = "DOMO DOMIN"
access_token = "DOMO Access token"

response_1 = requests.get(
    url=f"https://{domain}/api/query/v1/byos/accounts/basic",
    headers={"X-Domo-Developer-Token": access_token},
)

# Filter out the json containing 'friendlyName': '' - That contains the datasets in the database
cloud_id = {}
for cloud in response_1.json():
    if cloud['friendlyName'] == 'DOMO Multi-Cloud Connection Name':
        cloud_id = cloud['id']

print(cloud_id)

## Section 3 - Calling the Domo Dataset Outside of Domo

In [ ]:
from pydomo import Domo

import pandas as pd

client_id      = 'DOMO Client ID Found in using this url: https://developer.domo.com/ '
client_secret  = 'DOMO Client ID Found in using this url: https://developer.domo.com/ '

domo = Domo(client_id,client_secret,api_host='api.domo.com')

# Get the Dataset ID From and place in the URL

# Download a data set from Domo
df = domo.ds_get('Dataset url ID Dataset GUI (located in the URL when viewing the dataset view)')

df.head()

# Get the 'domo dataset id' column
dataset_id = df['Name of the Domo Dataset ID Column']
print(dataset_id)



## Section 3 - Using the ETL to get the Domo ID'S

In [ ]:
import domojupyter as domo
import pandas as pd

df = domo.read_dataframe('SnowFlake & Domo Mapping', query='SELECT * FROM table')
df.head()
# Get the 'domo dataset id' column
dataset_id = df['Domo Dataset ID']
print(dataset_id)

## Section 4 - Creating Iteration loop using the Dataset id and creating the mapping conditions 

In [ ]:
import requests

# Create a session object for making multiple requests
session = requests.Session()
session.headers.update({"X-Domo-Developer-Token": access_token})

for id in dataset_id:
    url = f"https://{domain}/api/query/v1/byos/accounts/{cloud_id}/datasets/{id}/mapping"
    

    try:
        response = session.get(url)
        response.raise_for_status()  # Raise an exception for any HTTP errors
        data = response.json()
        print(response.json())

    except requests.exceptions.RequestException as e:
        print(f"An error occurred for Dataset ID: {id}")
        print(f"Error details: {e}")
        print("---")


## Section 5 - Using the Domo table name column to use in the mapping in snowflake

In [ ]:
TABLENAME = df['Domo Table Name Colunn Name ']
print(TABLENAME)

## Section 6 - Creating the for loops that uses the mapping conditions and Name of the tables from the dataset to call the API - Change the Snowflake table names

In [ ]:
import requests
import time

# Create a session object for making multiple requests
session = requests.Session()
session.headers.update({
    "Content-Type": "application/json",
    "X-DOMO-Developer-Token": access_token,
})

# Define the delay between API calls (in seconds)
delay = 0.1

# Loop through each dataset ID
for i in range(len(dataset_id)):
    id = dataset_id[i]
    table_name = TABLENAME[i]

    url = f"https://{domain}/api/query/v1/byos/accounts/{cloud_id}/datasets/{id}/mapping"

    try:
        payload = {
            "mapping": {
                "tableName": table_name,
            },
            "standardize": True,
        }

        response = session.put(url, json=payload)
        response.raise_for_status()  # Raise an exception for any HTTP errors

        print("Mapping updated successfully")

    except requests.exceptions.RequestException as e:
        print(f"Error updating mapping for table '{table_name}' in dataset ID '{id}': {e}")

    # Pause for the specified delay before the next API call for the same dataset ID
    time.sleep(delay)


## Section 7 - Force the change in the API - Change the Snowflake table names

In [ ]:
import requests
import time

# Create a session object for making multiple requests
session = requests.Session()
session.headers.update({
    "Content-Type": "application/json",
    "X-DOMO-Developer-Token": access_token,
})

def update_index_for_dataset_id(dataset_id):
    url = f"https://{domain}/api/query/v1/datasources/{dataset_id}/index"

    try:
        payload = {
            "mapping": {
                "tableName": table_name,
            },
            "standardize": True,
        }

        response = session.post(url, json=payload)
        response.raise_for_status()  # Raise an exception for any HTTP errors

        if response.status_code == 200:
            print(response.json())
        else:
            print(f"Error updating index for dataset ID {dataset_id}: {response.status_code} - {response.reason}")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the request for dataset ID {dataset_id}: {str(e)}")

# Loop through each ID and call the function
for id in dataset_id:
    update_index_for_dataset_id(id)
    time.sleep(1)  # Pause for 1 second before the next API call
